In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[7]:


#!/usr/bin/env python
# coding: utf-8

# In[76]:


import numpy as np
import pandas as pd
import re 
import matplotlib.pyplot as plt 
import sys
import time


# In[77]:


#Filename =sys.argv[1] # Read input file 
df = pd.read_excel('sequenceInput.xlsx')


# In[78]:


df=df.dropna().reset_index(drop=True)
data= pd.DataFrame()
data=df

def get_integer_value(newdata):
    return re.findall('\d+',newdata)

# In[79]:


sulfide=df['Disulfide bond']
def get_sulfide_value(newdata):
    return re.findall('\d+\..\d+',newdata)

values= sulfide.apply(get_sulfide_value)
data['Disulfide bond']=values


# In[80]:
human_data=data[data['Organism'].str.contains("Homo sapiens")] #Selecting only organism which contains 'Homo sapiens'
data= human_data.reset_index(drop=True)

#Intrabond distance formation

lenB=pd.Series(data['Disulfide bond']).str.len()
intrabond=pd.Series(len(data),dtype=np.str)
for i in range(len(data)):
    intrabond[i]=''
j=0;
while j<len(data):
    for k in range(lenB[j]):
        split = re.findall('\d+',data['Disulfide bond'][j][k]) 
        firstPair= int(split[0])
        secondPair=int(split[1])
        intrabond[j]=intrabond[j]+str(secondPair-firstPair)+','    
    j=j+1
data['intrabond']=intrabond.apply(get_integer_value)

data['intrabond']=data['intrabond'].astype(str)
#Calculate average distance of intrabond distance
average= data['intrabond'].apply(get_integer_value)
lenIntra=average.str.len()

averageintrabond= np.zeros(len(data))
for i in range(len(data)):
    totalsum=0
    for j in range(lenIntra[i]):
        totalsum= totalsum+int(average[i][j])
    averageintrabond[i]=totalsum/lenIntra[i]
data['average_Intrabond']=averageintrabond


C:\Users\Amritpal Singh\Anaconda3\lib\site-packages\ipykernel_launcher.py:79: RuntimeWarning: invalid value encountered in longlong_scalars


In [2]:
def get_float_value(data):
    return re.findall('\d+\.\d+',data)

In [3]:
#Pair Score
array = pd.Series(data.size,dtype=np.str)
for x in range(data.size):
    array[x]=''

for i in range(0,len(data)):
    temp_array = np.zeros(lenB[i])
    for j in range(0,lenB[i]):
        for k in range(j+1,lenB[i]):
            if k!=lenB[i]:
                split1 = re.findall('\d+',data['Disulfide bond'][i][j]) 
                p1e1=int(split1[0])
                p1e2=int(split1[1])
                split2 = re.findall('\d+',data['Disulfide bond'][i][k]) 
                p2e1=int(split2[0])
                p2e2=int(split2[1])
            
                if(p2e1>p1e2):
                    temp_array[j]=temp_array[j]+0
                    temp_array[k]=temp_array[k]+0
                elif (p2e1<p1e2) and (p2e2>p1e2):
                    temp_array[j]=temp_array[j]+0.5
                    temp_array[k]=temp_array[k]+0.5    
                elif (p2e1<p1e2) and (p2e2<p1e2):
                    temp_array[j]=temp_array[j]+1.0
                    temp_array[k]=temp_array[k]+0.0
        array[i]=array[i]+str(temp_array[j])+','


In [4]:
data['Disulfide_score']= array.apply(get_float_value)

#Claculating disulfide pair score average 
disulfie_pair_score_sum = np.zeros(len(data),dtype=np.float)
for i in range(len(data)):
    for j in range(len(data['Disulfide_score'][i])):
        disulfie_pair_score_sum[i]= disulfie_pair_score_sum[i]+ float(data['Disulfide_score'][i][j])
data['disulfie_pair_score_sum']=disulfie_pair_score_sum

In [5]:
## Calculating Interbond Distance
interbond_distance= pd.Series(len(data),dtype= np.str)

for i in range(len(data)):
    interbond_distance[i]=''

for i in range(len(data)):
    
    if lenB[i]==0:
        interbond_distance[i]='No pair'
        
    if lenB[i]==1:
        interbond_distance[i]='Only One pair'
    
    for r in range(lenB[i]-1):
        split1= re.findall('\d+',data['Disulfide bond'][i][r])
        split2= re.findall('\d+',data['Disulfide bond'][i][r+1])
        interbond_distance[i]= interbond_distance[i]+ str(int(split2[0])-int(split1[1]))+ '|'
        
data['interbond_distance']=interbond_distance


In [6]:


# In[81]:


#Calculating length of sequences
Sequence_Length=pd.Series(data['Sequence']).str.len()
position = pd.Series(len(data),dtype=np.str)
for i in range(len(data)):
    position[i]=''


# In[82]:


for i in range(len(data)):
    for j in range((Sequence_Length[i])):
        if (data['Sequence'][i][j])=='C':
            position[i]=position[i]+','+str(j)


# In[83]:




data['Sequence C position']= position.apply(get_integer_value)
count =pd.Series(data['Sequence C position'])
length=count.str.len()
data['Sequence C count']=length


# In[84]:


pos = pd.Series(len(data),dtype=np.str)
for i in range(len(data)):
    pos[i]=''


for i in range(len(data)):
    for j in range(len(data['Sequence'][i])-2):
        if data['Sequence'][i][j]=='N' and data['Sequence'][i][j+1]!='P' and (data['Sequence'][i][j+2]=='S' or data['Sequence'][i][j+2]=='T'):
            pos[i]=pos[i]+str(j)+','
data['N_X_S/T']=pos.apply(get_integer_value)
data['Sequence N_X_S/T count']=pd.Series(data['N_X_S/T']).str.len()           


# In[86]:


data.to_excel('sequenceOutput.xlsx')


,Entry,Entry name,Status,Protein names,Gene names,Organism,Length,Sequence,Disulfide bond,intrabond,average_Intrabond,Disulfide_score,disulfie_pair_score_sum,interbond_distance,Sequence C position,Sequence C count,N_X_S/T,Sequence N_X_S/T count
0,P04439,HLAA_HUMAN,reviewed,"HLA class I histocompatibility antigen, A alph...",HLA-A HLAA,Homo sapiens (Human),365,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,"[125..188, 227..283]","['63', '56']",59.500000,"[0.0, 0.0]",0.0,39|,"[124, 187, 226, 282, 362]",5,[109],1
1,P01911,DRB_HUMAN,reviewed,"HLA class II histocompatibility antigen, DRB1 ...",HLA-DRB1,Homo sapiens (Human),266,MVCLKLPGGSCMTALTVTLMVLSSPLALSGDTRPRFLWQPKRECHF...,"[44..108, 146..202]","['64', '56']",60.000000,"[0.0, 0.0]",0.0,38|,"[2, 10, 43, 107, 145, 201]",6,[47],1
2,P01889,HLAB_HUMAN,reviewed,"HLA class I histocompatibility antigen, B alph...",HLA-B HLAB,Homo sapiens (Human),362,MLVMAPRTVLLLLSAALALTETWAGSHSMRYFYTSVSRPGRGEPRF...,"[125..188, 227..283]","['63', '56']",59.500000,"[0.0, 0.0]",0.0,39|,"[124, 187, 226, 282, 331, 348]",6,[109],1
3,P08246,ELNE_HUMAN,reviewed,Neutrophil elastase (EC 3.4.21.37) (Bone marro...,ELANE ELA2,Homo sapiens (Human),267,MTLGRRLACLFLACVLPALLLGGTALASEIVGGRRARPHAWPFMVS...,"[55..71, 151..208, 181..187, 198..223]","['16', '57', '6', '25']",26.000000,"[0.0, 1.5, 0.0, 0.5]",2.0,80|-27|11|,"[8, 13, 54, 70, 150, 180, 186, 197, 207, 222, ...",11,"[87, 123, 172]",3
4,P10144,GRAB_HUMAN,reviewed,Granzyme B (EC 3.4.21.79) (C11) (CTLA-1) (Cath...,GZMB CGL1 CSPB CTLA1 GRB,Homo sapiens (Human),247,MQPILLLLAFLLLPRADAGEIIGGHEAKPHSRPYMAYLMIWDQKSL...,"[49..65, 142..209, 173..188]","['16', '67', '15']",32.666667,"[0.0, 1.0, 0.0]",1.0,77|-36|,"[48, 64, 141, 172, 187, 208, 229]",7,"[70, 103]",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3583,A8MPY1,GBRR3_HUMAN,reviewed,Gamma-aminobutyric acid receptor subunit rho-3...,GABRR3,Homo sapiens (Human),467,MVLAFQLVSFTYIWIILKPNVCAASNIKMTHQRCSSSMKQTCKQET...,[184..198],['14'],14.000000,[0.0],0.0,Only One pair,"[21, 33, 41, 183, 197, 385, 412]",7,"[125, 196, 219]",3
3584,P28799,GRN_HUMAN,reviewed,Progranulin (PGRN) (Acrogranin) (Epithelin pre...,GRN,Homo sapiens (Human),593,MWTLVSWVALTAGLVAGTRCPDGQFCPVACCLDPGGASYSCCRPLL...,"[126..139, 133..149, 284..296, 290..306, 297.....","['13', '16', '12', '16', '17', '14', '13', '13...",13.916667,"[0.5, 0.5, 0.5, 1.0, 1.0, 1.0, 1.0, 0.5, 0.5, ...",8.0,-6|135|-6|-9|-7|-6|-6|31|-6|9|-6|,"[19, 25, 29, 30, 40, 41, 60, 66, 72, 82, 83, 9...",88,"[117, 235, 264, 367, 529]",5
3585,O00222,GRM8_HUMAN,reviewed,Metabotropic glutamate receptor 8 (mGluR8),GRM8 GPRC1H MGLUR8,Homo sapiens (Human),908,MVCEGKRSASCPCFFLLTAKFYWILTMMQRTHSQEYAHSIRVDGDI...,"[64..106, 246..534, 369..384, 424..431, 516..5...","['42', '288', '15', '7', '19', '18', '12', '13']",51.750000,"[0.0, 3.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0]",5.0,140|-165|40|85|-15|3|3|,"[2, 10, 12, 63, 105, 132, 245, 368, 383, 423, ...",28,"[94, 297, 451, 479, 564, 608, 891, 904]",8
3586,Q8IXB1,DJC10_HUMAN,reviewed,DnaJ homolog subfamily C member 10 (EC 1.8.4.-...,DNAJC10 ERDJ5 UNQ495/PRO1012,Homo sapiens (Human),793,MGVWLNKDDYIRDLKRIILCFLIVYMAILVGTDQDFYSLLGVSKTA...,"[158..161, 480..483, 588..591, 700..703]","['3', '3', '3', '3']",3.000000,"[0.0, 0.0, 0.0, 0.0]",0.0,319|105|109|,"[19, 157, 160, 185, 192, 263, 269, 293, 300, 4...",23,[529],1
